# **Clasificador de ropa de moda con TensorFlow**

En este bloc de notas se construirá un clasificador de ropa e introduciremos el uso de la biblioteca **Tensorflow** y uno de sus módulos más conocidos: **Keras**. Se utilizará uno de los datasets clásicos conocido como [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist).

In [ ]:
# TensorFlow y tf.keras
import tensorflow as tf
from tensorflow import keras

# Bibliotecas de ayuda
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

Importamos el dataset de Fashion MNIST

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

Este Dataset contiene imágenes en arreglos de Numpy de 28x28, donde cada elemento representa un color usando la escala RGB (0 a 255). Las etiquetas (labels) son un arreglo de enteros, que van de 0 a 9. Estos corresponden a la clase (class) de ropa que la imagen representa.

```
0	T-shirt/top (Playera/Top)
1	Trouser (Pantalón)
2	Pullover (Suéter ligero cerrado y con escote en pico)
3	Dress (Vestido)
4	Coat (Abrigo)
5	Sandal (Sandalia)
6	Shirt (Camisa)
7	Sneaker (Tenis)
8	Bag (Bolsa)
9	Ankle boot (Bota alta)
```


Cada imagen corresponde a una etiqueta en particular, crearemos el arreglo de clases ya que éste no es proporcionado por el dataset.

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

Exploremos el Dataset

In [ ]:
print(train_images.shape)
print(len(train_labels))

In [ ]:
print(train_labels)

In [ ]:
print(test_images.shape)
print(len(test_labels))

## **Pre-procesamiento**

Una vez que hemos cargado nuestro Dataset procederemos al siguiente paso: el pre-procesamiento o limpieza y estandarización de los datos. Como ya hemos mencionado antes, los colores siguen una escala de 0 a 255 por lo que típicamente cuando se trata de imágenes, usamos 255 para dividir cada valor y convertirlos a un valor entre 0 y 1.

In [ ]:
plt.figure()
plt.imshow(train_images[0], cmap="gray")
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

Vamos a verificar que el conjunto de datos se encuentre en el formato adecuado desplegando las primeras 10 imágenes del *training set* y desplegaremos el nombre de cada clase debajo de la imagen.

In [ ]:
print(train_images)

In [ ]:
plt.figure(figsize=(10,10))
for i in range(10):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

## **Construir el modelo**

Construir la red neuronal implica configurar las capas del modelo y posteriormente compilar el modelo.

### Configurando las capas

Los bloques de construcción básicos de una red neuronal son sus capas o *layers*.  La mayoría de las tareas del aprendizaje profundo (Deep learning) consiste en unir capas sencillas. La gran mayoría de los tipos de capas disponibles en la blibioteca Keras de Tensorflow tienen parámetros que se ajustarán (serán aprendidos) durante el entrenamiento.

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

La primera capa de esta red, `tf.keras.layers.Flatten`, transforma el formato de las imágenes de un arreglo bi-dimensional (de 28 por 28 pixeles) a un arreglo uni dimensional (de 28x28 pixeles = 784 pixeles). Una capa *Flatten* no tiene parámetros que aprender, simplemente nos sirve para reformatear el conjunto de datos.

Después de que los pixeles estan "aplanados", la secuencia consiste de dos capas `tf.keras.layers.Dense`. Estas están densamente conectadas, o completamente conectadas. La primera capa *Dense* tiene 128 nodos (o neuronas). La segunda (y última) capa es una capa de 10 nodos *softmax* que devuelve un arreglo de 10 probabilidades que suman a 1. Cada nodo contiene una calificacion que indica la probabilidad que la actual imagen pertenece a una de las 10 clases.

## **Compilando el modelo**

Antes de que el modelo esté listo para entrenar, se necesitan algunas configuraciones más. Éstas son agregadas durante el paso de compilación del modelo:

1.   *Loss function* (función de pérdida) —Esto mide que tan exacto es el modelo durante el entrenamiento. Quiere minimizar esta funcion para dirigir el modelo en la direccion adecuada.
2.   *Optimizer* (optimizador) — Esto es como el modelo se actualiza basado en el conjunto de datos que ve y la función de pérdida.
3.   *Metrics* (métricas) — Se usan para monitorear los pasos de entrenamiento y de pruebas. El siguiente ejemplo usa accuracy (exactitud), la fracción de la imágenes que son correctamente clasificadas.





In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Mostramos un resumen del modelo creado:

In [ ]:
model.summary()

## Entrenar el modelo

Entrenar el modelo de red neuronal requiere de los siguientes pasos:

1. Entregue los datos de entrenamiento al modelo. En este ejemplo, el conjunto de datos de entrenamiento está en los arreglos *train_images* y *train_labels*.
2. el modelo aprende a asociar imágenes y etiquetas.
3. Usted le pide al modelo que haga predicciones sobre un conjunto de datos que se encuentran en el ejemplo, incluido en el arreglo *test_images*. Verifique que las predicciones sean iguales a las etiquetas de el arreglo *test_labels*.

Para comenzar a entrenar, llame el metodo `model.fit`, es llamado así por que fit (ajusta) el modelo al conjunto de datos de entrenamiento:

In [ ]:
epocas = 10
model.fit(train_images, train_labels, epochs=epocas)

A medida que el modelo entrena, la pérdida y la exactitud son desplegadas. Este modelo alcanza una exactitud cercana a  0.90 (o 90%) sobre el conjunto de datos de entrenamiento.

## Evaluando la exactitud

A continuación comparamos el rendimiento del modelo sobre el conjunto de datos de prueba:

In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('\nPrueba de Exactitud (Test accuracy):', test_acc)

Como podemos ver la exactitud sobre el conjunto de datos de prueba es un poco menor que la exactitud sobre el conjunto de entrenamiento. Esta diferencia entre el entrenamiento (training) y el test se debe al overfitting (sobre ajuste). Sobre ajuste sucede cuando un modelo de aprendizaje máquina (ML) tiene un rendimiento menor sobre un conjunto de datos nuevo, que nunca antes ha visto comparado con el de entrenamiento.

## **Haciendo predicciones**

Habiendo entrenado el modelo, podemos usarlo para hacer predicciones.

In [ ]:
predictions = model.predict(test_images)

Con la línea de código anterior, hemos predecido la etiqueta de cada una de las imágenes del conjunto de prueba. Revisemos las predicciones:

In [ ]:
predictions[0]

Cada predicción es un arreglo de 10 números. Estos representan el nivel de "confianza" del modelo sobre las imágenes de cada uno de los 10 artículos de moda/ropa. Ustedes pueden revisar cual tiene el nivel más alto de confianza:

In [ ]:
np.argmax(predictions[0])

Entonces, el modelo tiene mayor confianza que esta imagen es un bota de tobillo "ankle boot" o class_names[9]. Examinando las etiquetas de test o de pruebas muestra que esta clasificación es correcta:

In [ ]:
test_labels[0]

Graficaremos el vector de predicciones para entender mejor lo anterior:

In [ ]:
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array, true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array, true_label[i]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

Revisemos la imagen [0] y la [12], sus predicciones y el arreglo de predicciones. Las etiquetas de predicción correctas están en azul y las incorrectas están en rojo. El número entrega el porcentaje (sobre 100) para la etiqueta predecida.

In [ ]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

In [ ]:
i = 5
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

Ahora se mostrarán varias imágenes con su predicciones. Nótese que el modelo puede arrojar una predicción incorrecta aún cuando tenga mucha certeza sobre su predicción.

In [ ]:
# Plot the first X test images, their predicted labels, and the true labels.
# Muestra las x primeras imágenes, sus etiquetas predecidas y los valores reales.
# Color correct predictions in blue and incorrect predictions in red.
# Las predicciones correctas están en azul y las incorrectas en rojo.
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()

Finalmente, usaremos el modelo entrenado para hacer una predicción sobre una única imagen.

In [ ]:
# Grab an image from the test dataset.
# Tomamos una imagen del dataset de pruebas.
img = test_images[1]

print(img.shape)

In [ ]:
plt.figure()
plt.imshow(img)
plt.colorbar()
plt.grid(False)
plt.show()

Los modelos de tf.keras son optimizados sobre batch o bloques, o colecciones de ejemplos por vez. De acuerdo a esto, aunque se use una única imagen hay agregarla a una lista:

In [ ]:
# Add the image to a batch where it's the only member.
# Agregamos la imagen a un lote donde es el único miembro
img = (np.expand_dims(img,0))

print(img.shape)

Hagamos la predicción:

In [ ]:
predictions_single = model.predict(img)

print(predictions_single)

In [ ]:
plot_value_array(1, predictions_single[0], test_labels)
_ = plt.xticks(range(10), class_names, rotation=45)

*model.predict* retorna una lista de listas para cada imagen dentro del batch o bloque de datos. Tomemos la predicción para nuestra única imagen dentro del batch o bloque:

In [ ]:
np.argmax(predictions_single[0])

El modelo predice una etiqueta de 2, que corresponde a la clase ubicada en esa posición.

In [ ]:
print(class_names[2])

## **Cierre**
Hemos llegado al final de la práctica, utilizamos el dataset de [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist) que contiene más de 70,000 imágenes en 10 categorías o clases. Las imágenes están en una resolución de 28x28 y muestran artículos de ropa.

<img src="https://tensorflow.org/images/fashion-mnist-sprite.png" style="width:650px;height:400px;">

Figura 1. Ejemplos de Fashion-MNIST (por Zalando, MIT License).




---



Respetando los derechos de autor, incluyo la declaración de Copyright ya que este ejemplo está basado en la documentación de TensorFlow disponible en: https://www.tensorflow.org/tutorials/keras/classification

In [ ]:
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.